In [5]:
from osgeo import ogr, osr
import os

In [6]:
driver = ogr.GetDriverByName('ESRI Shapefile')

inDataSet = driver.Open(os.path.join("assets", "taxi_zones", "taxi_zones.shp"))
inLayer = inDataSet.GetLayer()
# crs
inSpatialRef = inLayer.GetSpatialRef()
# spatialRef.ExportToWkt()

outSpatialRef = ogr.osr.SpatialReference()
outSpatialRef.ImportFromEPSG(3857)

coordTrans = ogr.osr.CoordinateTransformation(inSpatialRef, dest)

# create the output layer
outputShapefile = os.path.join("assets", "taxi_zones", "taxi_zones_mercator.shp")
if os.path.exists(outputShapefile):
    driver.DeleteDataSource(outputShapefile)
outDataSet = driver.CreateDataSource(outputShapefile)
outLayer = outDataSet.CreateLayer("basemap_3857", geom_type=ogr.wkbMultiPolygon)

# add fields
inLayerDefn = inLayer.GetLayerDefn()
for i in range(0, inLayerDefn.GetFieldCount()):
    fieldDefn = inLayerDefn.GetFieldDefn(i)
    outLayer.CreateField(fieldDefn)

# get the output layer's feature definition
outLayerDefn = outLayer.GetLayerDefn()

# loop through the input features
inFeature = inLayer.GetNextFeature()
while inFeature:
    # get the input geometry
    geom = inFeature.GetGeometryRef()
    # reproject the geometry
    geom.Transform(coordTrans)
    # create a new feature
    outFeature = ogr.Feature(outLayerDefn)
    # set the geometry and attribute
    outFeature.SetGeometry(geom)
    for i in range(0, outLayerDefn.GetFieldCount()):
        outFeature.SetField(outLayerDefn.GetFieldDefn(i).GetNameRef(), inFeature.GetField(i))
    # add the feature to the shapefile
    outLayer.CreateFeature(outFeature)
    # dereference the features and get the next input feature
    outFeature = None
    inFeature = inLayer.GetNextFeature()

# Save and close the shapefiles
inDataSet = None
outDataSet = None

In [ ]:
# this cell is for converting a point from from lat/long to web mercator
# this is separate from everything above.

from pyproj import Proj, transform

inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:3857')
lat_x, long_y = # somevalue, somevalue
merc_x, merc_y = transform(inProj, outProj, lat_x, lat_y)